In [1]:
import sys 
sys.path.append('/home/lumargot/trachoma/src/py')
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # put -1 to not use any
from torch import Tensor, nn
import torchvision

In [2]:
TORCHDYNAMO_VERBOSE=1
TORCH_LOGS="+dynamo"

In [3]:
from loaders.tt_dataset import BBXImageTestTransform, BBXImageEvalTransform
import SimpleITK as sitk
from nets.segmentation import FasterTTRCNN, MobileFasterTTRCNN
import torch
import numpy as np
from nets.segmentation import FasterTTRCNN
import matplotlib.pyplot as plt 
from matplotlib.patches import Rectangle

import torch
import torchvision.models as models
from torchvision.models.mobilenetv2 import MobileNet_V2_Weights
from executorch.backends.xnnpack.partition.xnnpack_partitioner import XnnpackPartitioner
from executorch.exir import to_edge_transform_and_lower

2025-07-02 08:21:34.687642: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751458894.700560   27976 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751458894.704596   27976 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751458894.715624   27976 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751458894.715637   27976 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751458894.715639   27976 computation_placer.cc:177] computation placer alr

ImportError: cannot import name 'MobileFasterTTRCNN' from 'nets.segmentation' (/home/lumargot/trachoma/src/py/nets/segmentation.py)

In [ ]:

ckpt = '/CMF/data/lumargot/trachoma/output/backtoold/5fold_v4/fold2/epoch=20-val_loss=1.83.ckpt'

In [ ]:
class TTUSegTorch(nn.Module):
    def __init__(self, faster, max_detections):
        super(TTUSegTorch, self).__init__()
        self.faster = faster
        self.max_detections = max_detections
        self.faster.eval()

    def forward(self, x):
        x = self.faster(x)
        num_detections = min(len(x[0]['boxes']), self.max_detections)

        boxes = torch.zeros((self.max_detections, 4), dtype=torch.float32)
        scores = torch.zeros(self.max_detections, dtype=torch.float32)
        labels = torch.zeros(self.max_detections, dtype=torch.float32)

        if num_detections > 0:
            boxes = x[0]['boxes'][:num_detections,:]
            scores = x[0]['scores'][:num_detections]
            labels = x[0]['labels'][:num_detections]
            
        return boxes, labels.float(), scores 
    


In [4]:
model = FasterTTRCNN(out_features=4, class_weights = torch.ones(4))

state_dict = torch.load(ckpt, weights_only=False, map_location='cpu') # or 'cuda' if loading to GPU

# Load the state dictionary into the model
model.load_state_dict(state_dict['state_dict'])

# Set the model to evaluation mode if you are using it for inference
model.eval()

NameError: name 'torch' is not defined

In [7]:
model_2 = TTUSegTorch(model.model, max_detections=25)
model_2.eval()

TTUSegTorch(
  (faster): FasterRCNN(
    (transform): GeneralizedRCNNTransform(
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        Resize(min_size=(800,), max_size=1333, mode='bilinear')
    )
    (backbone): BackboneWithFPN(
      (body): IntermediateLayerGetter(
        (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (layer1): Sequential(
          (0): Bottleneck(
            (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(64, eps=1e-05, mom

In [9]:

# model = models.mobilenetv2.mobilenet_v2(weights=MobileNet_V2_Weights.DEFAULT).eval()
sample_inputs = (torch.randn(1, 3, 384, 768), )

et_program = to_edge_transform_and_lower(
    torch.export.export(model_2, sample_inputs),
    partitioner=[XnnpackPartitioner()]
).to_executorch()

with open("model-example.pte", "wb") as f:
    f.write(et_program.buffer)

W0624 16:43:08.333000 2857941 /NIRAL/tools/anaconda3/envs/tt/lib/python3.10/site-packages/torch/fx/experimental/symbolic_shapes.py:6679] [1/0] failed during evaluate_expr(4*u1 > 4000, hint=None, size_oblivious=False, forcing_spec=False
E0624 16:43:08.336000 2857941 /NIRAL/tools/anaconda3/envs/tt/lib/python3.10/site-packages/torch/fx/experimental/recording.py:299] [1/0] failed while running evaluate_expr(*(4*u1 > 4000, None, False, False), **{})


UserError: Consider annotating your code using torch._check*(). Could not guard on data-dependent expression 4*u1 > 4000 (unhinted: 4*u1 > 4000).  (Size-like symbols: u1)

Caused by: if boxes.numel() > (4000 if boxes.device.type == "cpu" else 100_000) and not torchvision._is_tracing():  # torchvision/ops/boxes.py:73 in batched_nms (_dynamo/variables/tensor.py:1245 in evaluate_expr)
For more information, run with TORCH_LOGS="dynamic"
For extended logs when we create symbols, also add TORCHDYNAMO_EXTENDED_DEBUG_CREATE_SYMBOL="u1"
If you suspect the guard was triggered from C++, add TORCHDYNAMO_EXTENDED_DEBUG_CPP=1
For more debugging help, see https://docs.google.com/document/d/1HSuTTVvYH1pTew89Rtpeu84Ht3nQEFTYhAX3Ypa_xJs/edit?usp=sharing

User Stack (most recent call last):
  (snipped, see stack below for prefix)
  File "/tmp/ipykernel_2857941/1275767187.py", line 9, in forward
    x = self.faster(x)
  File "/tools/anaconda3/envs/tt/lib/python3.10/site-packages/torch/nn/modules/module.py